## Цель работы

&nbsp;&nbsp;&nbsp;&nbsp;Изучить постановку антагонистической игры двух лиц в нормальной форме; найти решение игры за обоих игроков в смешанных стратегиях (стратегическую седловую точку).

## Поставновка задачи

&nbsp;&nbsp;&nbsp;&nbsp;Для игры, заданной матрицей стратегий $c_{ij}$, требуется найти оптимальные смешанные стратегии обоих игроков, сведя матричную игру к задаче ЛП (прямой для одного игрока и двойственной для другого).

&nbsp;&nbsp;&nbsp;&nbsp;Задачи ЛП следует решать симплекс-методом, приводя начальные, промежуточные и конечные симплекс-таблицы. По окончании алгоритма полученные решения необходимо проверить на допустимость


## Ход работы

In [1]:
import numpy as np

from src.game import StrategyMatrix, optimal
from simplexlib.src.table import Table, Simplex, Format, pretty_value
from simplexlib.src.latex import matrix_to_latex
from IPython.display import display_markdown


matrix = StrategyMatrix(
    [
        [4, 4, 0, 6, 12],
        [1, 14, 14, 13, 11],
        [17, 6, 14, 4, 3],
        [18, 16, 13, 15, 16],
    ]
)

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Исходные данные согласно варианту:",
    matrix_to_latex(np.array(matrix.data, dtype=np.float64), "A", brackets="b"),
    raw=True
)

&nbsp;&nbsp;&nbsp;&nbsp;Исходные данные согласно варианту:

$A = \begin{bmatrix}
4 & 4 & 0 & 6 & 12\\
1 & 14 & 14 & 13 & 11\\
17 & 6 & 14 & 4 & 3\\
18 & 16 & 13 & 15 & 16
\end{bmatrix}$

In [2]:
source = matrix.inverse()

fmt = {
    "default_var_sym": "v",
    "default_tgt_sym": "W",
    "default_precision": 3,
}

source.data *= -1

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Найдем смешанные стратегии для игрока А. Сформулируем задачу для решения симплекс-методом:",
    Format(source, **fmt).target(),
    Format(source, **fmt).system(),
    Format(source, **fmt).var_zero_constraint(),
    "&nbsp;&nbsp;&nbsp;&nbsp;Исходная simplex-таблица:",
    Format(source, **fmt).table(),
    raw=True,
)

source.data *= -1

summary = Simplex.resolve(source)

for table, pos in zip(summary.history, summary.solvers):
    display_markdown(
        f"&nbsp;&nbsp;&nbsp;&nbsp;Позиция опорной точки: {pos}",
        Format(table, **fmt).table(),
        raw=True,
    )

g, vals = optimal(table)

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка решения:",
    Format(table, **fmt).check(summary.source.c),
    "&nbsp;&nbsp;&nbsp;&nbsp;Смешанная стратегия игрока А:",
    f"$g={pretty_value(g, 3)}$",
    f"$Y=[{', '.join(pretty_value(val, 3) for val in vals)}]$",
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка корректности решения:",
    f"${'+'.join(pretty_value(val, 3) for val in vals)} = {pretty_value(np.sum(vals), 3)}$",
    raw=True
)

&nbsp;&nbsp;&nbsp;&nbsp;Найдем смешанные стратегии для игрока А. Сформулируем задачу для решения симплекс-методом:

$W = v_1+v_2+v_3+v_4 \rightarrow min$

$\begin{cases}4v_1+v_2+17v_3+18v_4+v_5 = 1\\
4v_1+14v_2+6v_3+16v_4+v_6 = 1\\
14v_2+14v_3+13v_4+v_7 = 1\\
6v_1+13v_2+4v_3+15v_4+v_8 = 1\\
12v_1+11v_2+3v_3+16v_4+v_9 = 1
\end{cases}$

$v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8 ≥ 0$

&nbsp;&nbsp;&nbsp;&nbsp;Исходная simplex-таблица:

...

&nbsp;&nbsp;&nbsp;&nbsp;Смешанная стратегия игрока А:

$g=13.367$

$Y=[0, 0.267, 0.1, 0.633]$

&nbsp;&nbsp;&nbsp;&nbsp;Проверка корректности решения:

$0+0.267+0.1+0.633 = 1$

<p style="page-break-after:always;"></p>

In [3]:
source = matrix.straight()

summary = Simplex.resolve(source)

fmt = {
    "default_var_sym": "v",
    "default_tgt_sym": "Z",
    "default_precision": 3,
}

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Сформулируем задачу для нахождения оптимальной смешанной стратегии игрока B:",
    Format(source, **fmt).target(),
    Format(source, **fmt).system(),
    Format(source, **fmt).var_zero_constraint(),
    "&nbsp;&nbsp;&nbsp;&nbsp;Исходная таблица:",
    Format(source, **fmt).table(),
    raw=True,
)

for table, pos in zip(summary.history, summary.solvers):
    display_markdown(
        f"&nbsp;&nbsp;&nbsp;&nbsp;Позиция опорной точки: {pos}",
        Format(table, **fmt).table(),
        raw=True,
    )

h, vals = optimal(table)
vals *= -1

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка решения:",
    Format(table, **fmt).check(summary.source.c),
    "&nbsp;&nbsp;&nbsp;&nbsp;Смешанная стратегия игрока B:",
    f"$h={pretty_value(g, 3)}$",
    f"$X=[{', '.join(pretty_value(val, 3) for val in vals)}]$",
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка корректности решения:",
    f"${'+'.join(pretty_value(val, 3) for val in vals)} = {pretty_value(np.sum(vals), 3)}$",
    raw=True
)

&nbsp;&nbsp;&nbsp;&nbsp;Сформулируем задачу для нахождения оптимальной смешанной стратегии игрока B:

$Z = v_1+v_2+v_3+v_4+v_5 \rightarrow min$

$\begin{cases}4v_1+4v_2+6v_4+12v_5+v_6 = 1\\
v_1+14v_2+14v_3+13v_4+11v_5+v_7 = 1\\
17v_1+6v_2+14v_3+4v_4+3v_5+v_8 = 1\\
18v_1+16v_2+13v_3+15v_4+16v_5+v_9 = 1
\end{cases}$

$v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8 ≥ 0$

&nbsp;&nbsp;&nbsp;&nbsp;Исходная таблица:

...

&nbsp;&nbsp;&nbsp;&nbsp;Смешанная стратегия игрока B:

$h=13.367$

$X=[0.043, 0, 0.881, 0.076, 0]$

&nbsp;&nbsp;&nbsp;&nbsp;Проверка корректности решения:

$0.043+0+0.881+0.076+0 = 1$

## Вывод

&nbsp;&nbsp;&nbsp;&nbsp;В ходе работы была изучена постановка антагонистической игры двух лиц в нормальной форме и найдено решение игры за двух игроков в смешанных стратегиях, с помощью преобразования в ЗЛП и симплекс метода. Полученные смешанные стратегии были проверены на допустимость ($\sum x_i = 1$, $g = h$).

## Приложение

&nbsp;&nbsp;&nbsp;&nbsp;Исходный код ячейки с исходными данными:

In [ ]:
import numpy as np

from src.game import StrategyMatrix, optimal
from simplexlib.src.table import Table, Simplex, Format, pretty_value
from simplexlib.src.latex import matrix_to_latex
from IPython.display import display_markdown


matrix = StrategyMatrix(
    [
        [4, 4, 0, 6, 12],
        [1, 14, 14, 13, 11],
        [17, 6, 14, 4, 3],
        [18, 16, 13, 15, 16],
    ]
)

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Исходные данные согласно варианту:",
    matrix_to_latex(np.array(matrix.data, dtype=np.float64), "A", brackets="b"),
    raw=True
)

&nbsp;&nbsp;&nbsp;&nbsp;Исходный код ячейки первого игрока:

In [ ]:
source = matrix.inverse()

fmt = {
    "default_var_sym": "v",
    "default_tgt_sym": "W",
    "default_precision": 3,
}

source.data *= -1

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Найдем смешанные стратегии для игрока А. Сформулируем задачу для решения симплекс-методом:",
    Format(source, **fmt).target(),
    Format(source, **fmt).system(),
    Format(source, **fmt).var_zero_constraint(),
    "&nbsp;&nbsp;&nbsp;&nbsp;Исходная simplex-таблица:",
    Format(source, **fmt).table(),
    raw=True,
)

source.data *= -1

summary = Simplex.resolve(source)

for table, pos in zip(summary.history, summary.solvers):
    display_markdown(
        f"&nbsp;&nbsp;&nbsp;&nbsp;Позиция опорной точки: {pos}",
        Format(table, **fmt).table(),
        raw=True,
    )

g, vals = optimal(table)

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка решения:",
    Format(table, **fmt).check(summary.source.c),
    "&nbsp;&nbsp;&nbsp;&nbsp;Смешанная стратегия игрока А:",
    f"$g={pretty_value(g, 3)}$",
    f"$Y=[{', '.join(pretty_value(val, 3) for val in vals)}]$",
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка корректности решения:",
    f"${'+'.join(pretty_value(val, 3) for val in vals)} = {pretty_value(np.sum(vals), 3)}$",
    raw=True
)

&nbsp;&nbsp;&nbsp;&nbsp;Исходный код ячейки второго игрока:

In [ ]:
source = matrix.straight()

summary = Simplex.resolve(source)

fmt = {
    "default_var_sym": "v",
    "default_tgt_sym": "Z",
    "default_precision": 3,
}

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Сформулируем задачу для нахождения оптимальной смешанной стратегии игрока B:",
    Format(source, **fmt).target(),
    Format(source, **fmt).system(),
    Format(source, **fmt).var_zero_constraint(),
    "&nbsp;&nbsp;&nbsp;&nbsp;Исходная таблица:",
    Format(source, **fmt).table(),
    raw=True,
)

for table, pos in zip(summary.history, summary.solvers):
    display_markdown(
        f"&nbsp;&nbsp;&nbsp;&nbsp;Позиция опорной точки: {pos}",
        Format(table, **fmt).table(),
        raw=True,
    )

h, vals = optimal(table)
vals *= -1

display_markdown(
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка решения:",
    Format(table, **fmt).check(summary.source.c),
    "&nbsp;&nbsp;&nbsp;&nbsp;Смешанная стратегия игрока B:",
    f"$h={pretty_value(g, 3)}$",
    f"$X=[{', '.join(pretty_value(val, 3) for val in vals)}]$",
    "&nbsp;&nbsp;&nbsp;&nbsp;Проверка корректности решения:",
    f"${'+'.join(pretty_value(val, 3) for val in vals)} = {pretty_value(np.sum(vals), 3)}$",
    raw=True
)

&nbsp;&nbsp;&nbsp;&nbsp;Исходный код src/game.py:

In [4]:
# %load src/game.py
import numpy as np

from simplexlib.src.table import Table, V


class StrategyMatrix:
    data: np.ndarray

    def __init__(self, data: np.ndarray):
        self.data = data

    def straight(self) -> Table:
        return Table.straight(
            [1] * len(self.data[0]),
            *[V[line] <= 1 for line in self.data]
        )

    def inverse(self) -> Table:
        return Table.inverse(
            [1] * len(self.data[0]),
            *[V[line] <= 1 for line in self.data]
        )

def optimal(table: Table):
    h = 1 / table.F
    return h, table.real_b * h

&nbsp;&nbsp;&nbsp;&nbsp;Исходный код simplexlib/src/table.py:

In [5]:
# %load simplexlib/src/table.py
from __future__ import annotations

import inspect as insp
import numpy as np
import pandas as pd

from dataclasses import dataclass, field
from itertools import product
from functools import wraps
from typing import Any, Callable, Iterable, List, Optional

from .latex import array_to_latex, matrix_to_latex, equation_system, Brackets


def fill_defaults(method: Callable):
    """
    Helps to avoid writing duplicate code
    in each function with these arguments:

    ```
    precision = precision or self.default_precision
    var_sym = var_sym or self.default_var_sym
    tgt_sym = tgt_sym or self.default_tgt_sym
    ...
    ```
    """
    signature = insp.signature(method).parameters

    _, *signature = signature

    @wraps(method)
    def _wrapper(
        self: Table,
        *args,
        **kwargs,
    ):
        args = list(args)

        for idx, name in enumerate(signature):
            # Since self argument was skipped
            idx = idx + 1

            # Positional parameter
            if len(args) > idx and args[idx] is None:
                args[idx] = getattr(self, f"default_{name}", None)

            # Keyword parameter
            elif idx > len(args) and kwargs.get(name, None) is None:
                kwargs[name] = getattr(self, f"default_{name}", None)

        # Launch method execution
        return method(
            self,
            *args,
            **kwargs,
        )

    return _wrapper


@dataclass
class Vec:
    data: List

    def __le__(self, oth: float):
        return type(self)([*self.data, oth])

    def __ge__(self, oth: float):
        return type(self)([*self.data, -oth])

    @property
    def inv(self):
        self.data = [-elem for elem in self.data]

        return self


class V:
    """
    Allows V[elem1, elem2, ..., elemN] syntax on system definition
    """

    def __getitem__(self, key):
        return Vec(list(key))


V = V()


def pretty_coefficient(value: np.float64, precision: int) -> str:
    value = np.round(value, precision)

    if np.float64.is_integer(value):
        value = value.astype(int)

    return f"{'-' if value < 0 else ''}{'' if (val := np.abs(value)) == 1 else val.astype(str)}"


def pretty_value(value: np.float64, precision: int) -> str:
    value = np.round(value, precision)

    if np.float64.is_integer(value):
        value = value.astype(int)

    return value.astype(str)


def pretty_sum(array: Iterable[str]):
    return "+".join(array).replace("+-", "-")


@dataclass
class Format:
    # Source table reference
    victim: Table

    # Markdown output modifiers
    default_A_sym: str = "A"
    default_b_sym: str = "b"
    default_c_sym: str = "c"
    default_var_sym: str = "x"
    default_tgt_sym: str = "F"
    default_unbound: str = "s_0"
    default_precision: str = 2
    default_brackets: Brackets = Brackets.square

    @fill_defaults
    def system(
        self,
        precision: Optional[int] = None,
        var_sym: Optional[str] = None,
    ) -> str:
        """System as latex output"""
        # Create matrix (num of constraints X total number of variables)
        sq = np.zeros((len(self.victim.vlabels), self.victim.var_num))

        # Go along 0 axis of A
        for row, vec in enumerate(self.victim.A):
            # Variable is not present - set as +-1 depending on the constraint
            sq[row, self.victim.vlabels[row] - 1] = -1 if self.victim.b[row] < 0 else 1

            # Go along 1 axis of A
            for elem, col in zip(vec, self.victim.hlabels):
                # Set respective variables
                sq[row, col - 1] = elem

        # Constraints are positive
        constr = np.abs(self.victim.b)
        # Labels are sorted in ascention order
        labels = [f"{var_sym}_{idx + 1}" for idx in range(self.victim.var_num)]

        # Build equation system
        return equation_system(
            sq,
            constr,
            labels,
            precision,
        )

    @fill_defaults
    def target(
        self,
        precision: Optional[int] = None,
        var_sym: Optional[str] = None,
        tgt_sym: Optional[str] = None,
    ) -> str:
        """Target vector as latex output"""
        # Cast c to pretty strings
        c = [pretty_coefficient(elem, precision) for elem in self.victim.c]
        # Make equation
        eq = pretty_sum(
            [f"{elem}{var_sym}_{label}" for elem, label in zip(c, self.victim.hlabels)]
        )

        # Return target equation
        return f"${tgt_sym} = {eq} \\rightarrow {'min' if self.victim.minimize else 'max'}$"

    @fill_defaults
    def table(
        self,
        precision: Optional[int] = None,
        var_sym: Optional[str] = None,
        tgt_sym: Optional[str] = None,
        unbound: Optional[str] = None,
    ) -> str:
        """System table as latex output"""
        return pd.DataFrame(
            np.round(self.victim.data, precision),
            index=[f"${var_sym}_{value}$" for value in self.victim.vlabels]
            + [f"${tgt_sym}$"],
            columns=[f"${unbound}$"]
            + [
                f"${var_sym}_{value}$"
                for value in (
                    self.victim.hlabels
                    if not self.victim.expanded
                    else [idx + 1 for idx, _ in enumerate(self.victim.hlabels)]
                )
            ],
        ).to_markdown()

    @fill_defaults
    def A(
        self,
        precision: Optional[int] = None,
        A_sym: Optional[str] = None,
        brackets: Optional[Brackets] = None,
    ) -> str:
        """A matrix as latex output"""
        return matrix_to_latex(self.victim.A, A_sym, precision, brackets)

    @fill_defaults
    def b(
        self,
        precision: Optional[int] = None,
        b_sym: Optional[str] = None,
        brackets: Optional[Brackets] = None,
    ) -> str:
        """b array as latex output"""
        return array_to_latex(self.victim.b, b_sym, precision, brackets)

    @fill_defaults
    def c(
        self,
        precision: Optional[int] = None,
        c_sym: Optional[str] = None,
        brackets: Optional[Brackets] = None,
    ) -> str:
        """c array as latex output"""
        return array_to_latex(self.victim.c, c_sym, precision, brackets)

    @fill_defaults
    def var_zero_constraint(self, var_sym: Optional[str] = None) -> str:
        """Variables with greater-or-equal constraint"""
        # lower-to-greater ordered variables
        vars = [f"{var_sym}_{idx}" for idx in range(self.victim.var_num)]
        # With >= 0 constraint
        res = ", ".join(vars) + " ≥ 0"
        return f"${res}$"

    @fill_defaults
    def base_vars(
        self,
        var_sym: Optional[str] = None,
        precision: Optional[int] = None,
    ) -> str:
        b = [pretty_value(elem, precision) for elem in self.victim.b]

        eq = ", ".join(
            f"{var_sym}_{label} = {value}"
            for value, label in zip(b, self.victim.vlabels)
        )

        return f"${eq}$"

    @fill_defaults
    def free_vars(
        self,
        var_sym: Optional[str] = None,
    ) -> str:
        eq = " = ".join(f"{var_sym}_{label}" for label in self.victim.hlabels)

        return f"${eq} = 0$"

    @fill_defaults
    def solution(
        self,
        var_sym: Optional[str] = None,
        precision: Optional[int] = None,
    ) -> str:
        eq = ", ".join(
            f"{var_sym}_{label} = {pretty_value(value, precision)}"
            for value, label in zip(self.victim.real_b, self.victim.rlabels)
        )

        return f"${eq}$"

    @fill_defaults
    def check(
        self,
        against: np.ndarray,
        zeros: bool = False,
        var_sym: Optional[str] = None,
        tgt_sym: Optional[str] = None,
        precision: Optional[str] = None,
    ) -> str:
        # Create string representation of real labels
        labels = [f"{var_sym}_{label}" for label in self.victim.rlabels]
        scoeffs = [pretty_coefficient(value, precision) for value in self.victim.real_b]
        sscoeffs = [pretty_value(value, precision) for value in self.victim.real_b]
        sagainst = [pretty_value(value, precision) for value in against]

        # Make line with coefficients
        fline = pretty_sum(
            f"{coef}{label}"
            for coef, label, rcoef in zip(scoeffs, labels, self.victim.real_b)
            if not zeros or rcoef != 0
        )

        # Make line with sum
        sline = pretty_sum(
            f"{coef}\cdot{f'({value})' if rvalue < 0 else value}"
            for coef, value, rcoef, rvalue in zip(
                sscoeffs, sagainst, self.victim.real_b, against
            )
            if not zeros or (rcoef != 0 and rvalue != 0)
        )

        # Compute result array and cast it to strings
        result = self.victim.real_b * against
        sresult = [
            pretty_value(value, precision)
            for value in result
            if not zeros or value != 0
        ]
        tline = pretty_sum(sresult)

        rresult = self.victim.result * (-1 if self.victim.minimize else 1)
        return f"${tgt_sym} = {pretty_value(rresult, precision)} = {fline} = {sline} = {tline} = {pretty_value(np.sum(result), precision)}$"


class Table:
    # Actual system data
    data: np.ndarray
    minimize: bool = True
    expanded: bool = False

    # Rows & columns mapping
    hlabels: list
    vlabels: list

    # "Real" labels
    rlabels: list

    def __init__(
        self,
        c: np.ndarray,
        A: np.ndarray,
        b: np.ndarray,
        vlabels: List[int],
        hlabels: List[str],
        rlabels: Optional[List[str]] = None,
        F: np.float64 = 0.0,
        minimize: bool = True,
        expanded: bool = False,
    ):
        """Creates simplex table data"""
        # Create simplex table
        constr = np.expand_dims(b, axis=1)
        data = np.hstack((constr, A))
        tgt = np.hstack((F, c))

        # Set result table
        self.data = np.vstack((data, tgt))

        # Set labels
        self.vlabels = vlabels
        self.hlabels = hlabels
        self.rlabels = rlabels or list(hlabels)

        # Set opt target
        self.minimize = minimize
        self.expanded = expanded

    @property
    def A(self) -> np.ndarray:
        """A input matrix"""
        return self.data[:-1, 1:]

    @property
    def b(self) -> np.ndarray:
        """b input array"""
        return self.data[:-1, 0]

    @property
    def c(self) -> np.ndarray:
        """c input array"""
        return self.data[-1, 1:]

    @property
    def s0(self) -> np.ndarray:
        """
        s0 column with free coefficients
        (idk if this is the way to call s0)
        """
        return self.b

    @property
    def F(self) -> np.ndarray:
        """Result value"""
        return self.data[-1, 0]

    @property
    def result(self) -> np.ndarray:
        """Result value alias"""
        return self.F

    @property
    def real_b(self) -> np.ndarray:
        """values of source variables"""
        current = {label: value for value, label in zip(self.b, self.vlabels)}
        values = [current.get(label, np.float64(0)) for label in self.rlabels]

        return np.array(values)

    @property
    def var_num(self) -> int:
        """Number of bound/unbound variables"""
        return (
            len(self.hlabels)
            if self.expanded
            else (len(self.hlabels) + len(self.vlabels))
        )

    @classmethod
    def straight(
        cls, target: Iterable[float], *system: List[Vec], expanded: bool = False
    ) -> Table:
        """
        Place for logic & validation
        To be used with Vec objects
        """
        system = [(v.data[:-1], v.data[-1]) for v in system]
        A, b = list(map(list, zip(*system)))
        labels = list(range(1, len(A[0]) + len(A) + 1))

        if expanded:
            A = np.hstack((A, np.eye(len(A), len(A), dtype=np.float64)))
            target = np.hstack((target, np.zeros((len(A),), dtype=np.float64)))

            return cls(
                target,
                A,
                np.array(b, dtype=np.float64),
                labels[len(b) :],
                labels,
                labels[: len(b)],
                expanded=expanded,
            )

        return cls(
            np.array(target, dtype=np.float64),
            np.array(A, dtype=np.float64),
            np.array(b, dtype=np.float64),
            labels[len(A[0]) :],
            labels[: len(A[0])],
        )

    @classmethod
    def inverse(
        cls,
        target: Iterable[float],
        *system: List[Vec],
    ) -> Table:
        system = [(v.data[:-1], v.data[-1]) for v in system]
        A, b = list(map(list, zip(*system)))
        labels = list(range(1, len(A[0]) + len(A) + 1))

        return cls(
            np.array(b, dtype=np.float64).T * -1,
            np.array(A, dtype=np.float64).T * -1,
            np.array(target, dtype=np.float64).T * -1,
            labels[len(A) :],
            labels[: len(A)],
        )

    def __rshift__(self, oth: Callable):
        """
        Allows to set optimization target in functional style:
        ```
        Table >> (min | max)
        ```
        """
        # If -> min
        if oth is min:
            # Check if target is not min, otherwise leave F as is
            if not self.minimize:
                self.data[-1, :] *= -1

                self.minimize = True

            return self

        # If -> max
        if oth is max:
            # Check if target is min, otherwise leave F as is
            if self.minimize:
                # multiply F vector by -1
                self.data[-1, :] *= -1

                self.minimize = False

            return self

        # Other values are not allowed
        raise ValueError("Only `min` & `max` funtions are allowed for `>` use")

    def __getitem__(self, item: Any) -> np.ndarray:
        # Forward __getitem__ to underlying table
        return self.data.__getitem__(item)

    def __setitem__(self, key: Any, value: Any) -> None:
        # Forward __setitem__ to underlying table
        return self.data.__setitem__(key, value)

    def add_constraint(
        self,
        equation: Vec,
    ) -> Table:
        """
        Adds constraint to the system, returns new Table
        To be used with Vec object
        """
        c = self.c.copy()
        if self.expanded:
            c = np.hstack((c, [0]))

        A = np.vstack((self.A.copy(), equation.data[:-1]))
        if self.expanded:
            expansion = np.zeros((len(A),))
            expansion[-1] = 1.0
            expansion = np.expand_dims(expansion, axis=1)
            A = np.hstack((A, expansion))

        b = np.hstack((self.b.copy(), equation.data[-1]))

        # Copy table with one more line to the system
        return Table(
            c,
            A,
            b,
            self.vlabels + [self.var_num + 1],
            self.hlabels + ([self.var_num + 1] if self.expanded else []),
            self.rlabels,
            self.F,
            self.minimize,
            self.expanded,
        )

    @property
    def clone(self) -> Table:
        """Convinience method for preserving table state"""

        return Table(
            self.c.copy(),
            self.A.copy(),
            self.b.copy(),
            list(self.vlabels),
            list(self.hlabels),
            list(self.rlabels),
            self.F,
            self.minimize,
            self.expanded,
        )


@dataclass
class SimplexResult:
    fixed: bool = field(default=True)
    solved: bool = field(default=False)
    fix_history: list = field(default_factory=list)
    fix_pos: list = field(default_factory=list)
    sol_history: list = field(default_factory=list)
    sol_pos: list = field(default_factory=list)

    def __bool__(self):
        return self.fixed and self.solved

    @property
    def history(self):
        return self.fix_history[1:] + self.sol_history[1:]

    @property
    def solvers(self):
        return self.fix_pos + self.sol_pos

    @property
    def source(self):
        if not self.fix_history:
            raise ValueError("Incomplete result")

        return self.fix_history[0]

    @property
    def result(self):
        if not self.sol_history:
            raise ValueError("Incomplete result")

        return self.sol_history[-1]


class Simplex:
    @classmethod
    def _find_negative(cls, table: Table, idx: int) -> Optional[int]:
        # Values of non-basis variables in row of negative s0
        row = table[idx, 1:].flatten()

        # Look for first negative
        idx = np.argmin(row)

        # Extract index (compensate for missing leading value by adding one to index)
        return (idx + 1) if row[idx] < 0 else None

    @classmethod
    def _find_pivot(cls, table: Table) -> Optional[int]:
        # Values of non-basis variables in F-row (w/o s0-column value)
        row = table.c.flatten()

        # Look for first positive/negative
        idx = (np.argmax if table.minimize else np.argmin)(row)

        # Extract index (compensate for missing leading value by adding one to index)
        return (idx + 1) if (row[idx] > 0 if table.minimize else row[idx] < 0) else None

    @classmethod
    def _find_minimal_ratio(
        cls,
        table: Table,
        jdx: int,
    ) -> Optional[int]:
        # s0 & solver columns
        column = table.b.flatten()
        solver_column = table[:-1, jdx].flatten()

        # I'll map division results to array of +np.inf
        # so any values, which either break divison (or division result < 0)
        # will end up as +np.inf and do not mess up argmin
        infs = np.full_like(column, +np.inf)
        idx = np.argmin(
            np.divide(
                column,
                solver_column,
                out=infs,
                where=(solver_column != 0) & (column * solver_column >= 0),
            )
        )

        # Check if there were any valid ratios & return idx
        return idx if infs[idx] != +np.inf else None

    @classmethod
    def _find_fixer(cls, table: Table) -> tuple[tuple[int, int], bool]:
        # Values of s0-column (w/o F-row value)
        column = table[:-1, 0].flatten()

        # Look for first negative
        indices = np.flatnonzero(column < 0)

        # Extract index
        idx = indices[0] if len(indices) else None

        if idx is None:
            return None, True

        jdx = cls._find_negative(table, idx)

        # No column - either solved, or not solvable at all
        if jdx is None:
            return None, False

        # Find minimal positive ratio of solver s0 & solver column
        idx = cls._find_minimal_ratio(table, jdx) if jdx else None

        # There is no solver row
        if idx is None:
            return None, False

        # Return tuple if both of indices are valid
        return (idx, jdx), True

    @classmethod
    def _find_solver(cls, table: Table) -> tuple[tuple[int, int], bool]:
        jdx = cls._find_pivot(table)

        # No column - solved
        if jdx is None:
            return None, True

        # Find minimal positive ratio of solver s0 & solver column
        idx = cls._find_minimal_ratio(table, jdx) if jdx else None

        # There is no solver row
        if idx is None:
            return None, False

        # Return tuple if both of indices are valid
        return (idx, jdx), True

    @classmethod
    def _step(cls, table: Table, idx: int, jdx: int):
        # Table dimensions
        x, y = table.data.shape

        # Adjust labels. Sub 1 from jdx since labels do not include s0
        table.vlabels[idx], table.hlabels[jdx - 1] = (
            table.hlabels[jdx - 1],
            table.vlabels[idx],
        )

        # Solver-values shortcut
        solver = table[idx, jdx]

        # Flatten makes copies
        solver_row = table[idx, :].flatten()
        solver_column = table[:, jdx].flatten()

        # Is there any other way?
        if not table.expanded:
            for i, j in product(range(x), range(y)):
                if i == idx and j == jdx:
                    table[i, j] = 1 / solver
                elif i == idx:
                    table[i, j] /= solver
                elif j == jdx:
                    table[i, j] /= -solver
                else:
                    table[i, j] -= solver_column[i] * solver_row[j] / solver
        else:
            for i, j in product(range(x), range(y)):
                # if i == idx and j == jdx:
                #     table[i, j] = 1 / solver
                if i == idx:
                    table[i, j] /= solver
                # elif j == jdx:
                #     table[i, j] /= -solver
                else:
                    table[i, j] -= solver_column[i] * solver_row[j] / solver

        return table

    @classmethod
    def fix(cls, table: Table):
        # First history entry is always source table
        history, positions = [table], list()
        while fixer := cls._find_fixer(table):
            # Dispatch _find_fixer result
            pos, fixable = fixer

            # Table wont fix
            if not fixable:
                fixed = False

                break

            # Fixable & no fix position - fixed
            if not pos:
                fixed = True

                break

            # Perform matrix transofrm
            table = cls._step(table.clone, *pos)

            # Write resulting table to history
            history.append(table)
            positions.append(pos)

        return history, positions, fixed

    @classmethod
    def solve(cls, table: Table):
        # First history entry is always source table
        history, positions = [table], list()
        while solver := cls._find_solver(table):
            # Dispatch _find_solver result
            pos, solvable = solver

            # Table wont solve
            if not solvable:
                solved = False

                break

            # Solvable & no solve position - solved
            if not pos:
                solved = True

                break

            # Perform matrix transofrm
            table = cls._step(table.clone, *pos)

            # Write resulting table to history
            history.append(table)
            positions.append(pos)

        return history, positions, solved

    @classmethod
    def resolve(cls, table: Table) -> SimplexResult:
        # Trying to fix input table
        fix_hist, fix_pos, fixed = cls.fix(table)

        # Unable to fix input table
        if not fixed:
            return SimplexResult(fixed, False, fix_hist, fix_pos)

        # The last table is resulting fix-table
        *_, table = fix_hist

        # Tryin to solve fixed table
        sol_hist, sol_pos, solved = cls.solve(table)

        return SimplexResult(fixed, solved, fix_hist, fix_pos, sol_hist, sol_pos)


ImportError: attempted relative import with no known parent package

&nbsp;&nbsp;&nbsp;&nbsp;Исходный код simplexlib/src/latex.py:

In [ ]:
# %load simplexlib/src/latex.py
import numpy as np

from enum import Enum


class Brackets(str, Enum):
    plain = ""
    round = "p"
    square = "b"
    curly = "B"
    pipes = "v"
    double = "V"


def array_to_latex(
    array: np.ndarray,
    name: str = str(),
    precision: int = 2,
    brackets: Brackets = Brackets.plain,
):
    assert len(array.shape) == 1, f"Can not display array of shape: {array.shape}"

    if isinstance(brackets, str):
        brackets = Brackets(brackets)

    array = np.round(array, precision)
    row = " & ".join(
        str(value.astype(int)) if np.float64.is_integer(value) else value.astype(str)
        for value in array
    )
    line = (
        name
        and f"{name} = "
        + f"\\begin{{{brackets.value}matrix}}\n{row}\n\\end{{{brackets.value}matrix}}"
    )

    return f"${line}$"


def matrix_to_latex(
    array: np.ndarray,
    name: str = str(),
    precision: int = 2,
    brackets: Brackets = Brackets.plain,
):
    assert len(array.shape) == 2, f"Can not display array of shape: {array.shape}"

    if isinstance(brackets, str):
        brackets = Brackets(brackets)

    array = np.round(array, precision)
    rows = "\\\\\n".join(
        " & ".join(
            str(value.astype(int))
            if np.float64.is_integer(value)
            else value.astype(str)
            for value in row
        )
        for row in array
    )
    line = (
        name
        and f"{name} = "
        + f"\\begin{{{brackets.value}matrix}}\n{rows}\n\\end{{{brackets.value}matrix}}"
    )

    return f"${line}$"


def format_value(val: np.float64) -> str:
    if np.float64.is_integer(val):
        val = val.astype(int)

    absed = np.abs(val)

    return ("-" if val < 0 else "") + (absed.astype(str) if absed != 1 else "")


def equation_body(values: list, labels: list, precision: int = 3) -> str:
    return "+".join(
        f"{format_value(value)}{var}"
        for var, val in zip(labels, values)
        if (value := np.round(val, precision))
    ).replace("+-", "-")


def equation_system(
    values: np.ndarray, result: np.ndarray, labels: np.array, precision: int = 3
) -> str:
    line = "\\\\\n".join(
        f"{equation_body(val, labels, precision)} = {res.astype(int) if np.float64.is_integer(res) else res}"
        for val, res in zip(values, result)
    )

    return f"$\\begin{{cases}}{line}\n\\end{{cases}}$"


def check(left: np.ndarray, right: np.ndarray, precision: int = 3) -> str:
    result = sum(left * right)

    # Округляем все значения
    left, right = [
        [np.round(value, precision) for value in values] for values in [left, right]
    ]

    # Соединяем при помощи cdot все пары значений без 0
    values = [
        "\cdot".join([f"({val})" if val < 0 else str(val) for val in values])
        for values in zip(left, right)
        if all(values)
    ]

    # Все суммируем, добавляем результат
    return "+".join(values) + f"={np.round(result, precision)}"
